In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

import json
import pandas as pd
import io
import requests
from collections import defaultdict

In [2]:
min_health = pd.read_csv('../Ministry_of_Health_Public_Health_Unit_Boundary.csv')

In [3]:
min_health

,OGF_ID,PHU_ID,PHU_NAME_ENG,PHU_NAME_FR,GEOMETRY_UPDATE_DATETIME,EFFECTIVE_DATETIME,SYSTEM_DATETIME,OBJECTID
0,71878062,2270,York Region Public Health,Service de santé de la région de York,1970/01/01 00:00:00+00,2020/05/15 12:56:15+00,2020/05/15 22:17:15+00,321
1,71878005,5183,Huron Perth Health Unit,Circonscription Sanitaire de Huron et Perth,2020/05/15 13:03:57+00,2020/05/15 12:54:50+00,2020/05/15 22:17:15+00,337
2,71878059,2265,"Region of Waterloo, Public Health",Bureau de santé de la région de Waterloo,1970/01/01 00:00:00+00,2020/05/15 12:55:42+00,2020/05/15 22:17:15+00,353
3,71878029,4913,Southwestern Public Health,Bureau de santé du Sud-Ouest,2020/05/15 13:03:57+00,2020/05/15 12:52:27+00,2020/05/15 22:17:15+00,369
4,71878057,2237,Hamilton Public Health Services,Services de santé publique de Hamilton,1970/01/01 00:00:00+00,2020/05/15 12:55:24+00,2020/05/15 22:17:15+00,385
5,71878051,2262,Thunder Bay District Health Unit,Bureau de santé du district de Thunder Bay,1970/01/01 00:00:00+00,2020/05/15 12:59:18+00,2020/05/15 22:17:15+00,401
6,71878041,2253,Peel Public Health,Bureau de santé de la région de Peel,1970/01/01 00:00:00+00,2020/05/15 12:56:08+00,2020/05/15 22:17:15+00,417
7,71878038,2242,Lambton Public Health,Comté de Lambton,1970/01/01 00:00:00+00,2020/05/15 12:51:11+00,2020/05/15 22:17:15+00,433
8,71878035,2266,Wellington-Dufferin-Guelph Health Unit,Bureau de santé de Wellington-Dufferin-Guelph,1970/01/01 00:00:00+00,2020/05/15 12:55:52+00,2020/05/15 22:17:15+00,449
9,71878032,2227,Brant County Health Unit,Bureau de santé du comté de Brant,1970/01/01 00:00:00+00,2020/05/15 12:55:34+00,2020/05/15 22:17:15+00,465


In [4]:
df = pd.read_csv('../Ontario_PHU.csv')

In [5]:
#ACTIVE_CASES, RESOLVED_CASES, and DEATHS are all cumulative based on the date. The number of deaths up to today.

In [8]:
df.loc[df['PHU_NAME']=='HALTON REGION']

,_id,FILE_DATE,PHU_NAME,PHU_NUM,ACTIVE_CASES,RESOLVED_CASES,DEATHS
1,2,20200410,HALTON REGION,2236,112,137,9
35,36,20200411,HALTON REGION,2236,109,147,10
69,70,20200412,HALTON REGION,2236,111,154,10
103,104,20200413,HALTON REGION,2236,118,162,11
137,138,20200414,HALTON REGION,2236,148,171,11
...,...,...,...,...,...,...,...
9700,9701,20210110,HALTON REGION,2236,1129,5443,131
9734,9735,20210111,HALTON REGION,2236,1106,5546,132
9768,9769,20210112,HALTON REGION,2236,1043,5655,133
9802,9803,20210113,HALTON REGION,2236,1021,5762,136


In [9]:
df.loc[df['PHU_NAME']=='PEEL REGION']

,_id,FILE_DATE,PHU_NAME,PHU_NUM,ACTIVE_CASES,RESOLVED_CASES,DEATHS
12,13,20200410,PEEL REGION,2253,575,222,14
46,47,20200411,PEEL REGION,2253,603,257,15
80,81,20200412,PEEL REGION,2253,636,284,17
114,115,20200413,PEEL REGION,2253,648,323,18
148,149,20200414,PEEL REGION,2253,683,343,19
...,...,...,...,...,...,...,...
9711,9712,20210110,PEEL REGION,2253,5572,38950,487
9745,9746,20210111,PEEL REGION,2253,5617,39436,487
9779,9780,20210112,PEEL REGION,2253,5614,39984,487
9813,9814,20210113,PEEL REGION,2253,5549,40574,498


In [10]:
len(df['PHU_NAME'].unique())

34

In [11]:
len(df['PHU_NUM'].unique())

34

In [12]:
def build_phu_map(df):
    df = df.groupby(['PHU_NUM', 'PHU_NAME']).size().reset_index()
    df.set_index(['PHU_NUM'],inplace=True)
    return df
hm = build_phu_map(df)
hm

,PHU_NAME,0
PHU_NUM,,
2226,ALGOMA DISTRICT,290
2227,BRANT COUNTY,290
2230,DURHAM REGION,290
2233,GREY BRUCE,290
2234,HALDIMAND-NORFOLK,290
2235,"HALIBURTON, KAWARTHA, PINE RIDGE",290
2236,HALTON REGION,290
2237,CITY OF HAMILTON,290
2238,HASTINGS & PRINCE EDWARD COUNTIES,290


In [13]:
def cumulative_count(df):
    hm=defaultdict(int)
    for index, row in df.iterrows():
        hm[row['PHU_NUM']]+=1
        df.loc[index, 'cum_count'] = hm[row['PHU_NUM']]
    return df

In [14]:
def get_most_recent_day(df):
    #grab only the rows for each phu on the most recent day
    df.sort_values(by='FILE_DATE',ascending=False,inplace=True)
    df = cumulative_count(df)
    return df.loc[df['cum_count']==1]
recent_df = get_most_recent_day(df)
recent_df.reset_index(drop=True)

,_id,FILE_DATE,PHU_NAME,PHU_NUM,ACTIVE_CASES,RESOLVED_CASES,DEATHS,cum_count
0,9860,20210114,YORK REGION,2270,2465,19010,423,1.0
1,9834,20210114,GREY BRUCE,2233,70,505,0,1.0
2,9841,20210114,LAMBTON COUNTY,2242,217,1217,28,1.0
3,9840,20210114,"KINGSTON, FRONTENAC, LENNOX & ADDINGTON",2241,45,576,1,1.0
4,9839,20210114,HURON PERTH,5183,123,808,25,1.0
5,9838,20210114,HASTINGS & PRINCE EDWARD COUNTIES,2238,20,306,5,1.0
6,9837,20210114,HALTON REGION,2236,973,5894,137,1.0
7,9835,20210114,HALDIMAND-NORFOLK,2234,136,944,41,1.0
8,9833,20210114,EASTERN ONTARIO,2258,446,1505,46,1.0
9,9843,20210114,MIDDLESEX-LONDON,2244,1186,3389,91,1.0


In [24]:
phu_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-79.17031943375656, 43.855456530375854],
      [-79.32800131723552, 44.22332648698636],
      [-79.15610036565454, 44.26317450192772],
      [-79.2424520113459, 44.45979253175579],
      [-79.5089783935667, 44.4223653127206],
      [-79.51627177236196, 44.16587376818996],
      [-79.54124177857526, 44.146300563626234],
      [-79.55698107718413, 44.07557695063855],
      [-79.63702649779411, 44.02061913733245],
      [-79.77551993922904, 43.989793125433756],
      [-79.63922387414868, 43.749907390083074],
      [-79.17031943375656, 43.855456530375854]]]},
   'properties': {'OGF_ID': 71878062,
    'PHU_ID': 2270,
    'PHU_NAME_E': 'York Region Public Health',
    'PHU_NAME_F': 'Service de santé de la région de York',
    'GEOMETRY_U': '1970-01-01T00:00:00.000Z',
    'EFFECTIVE_': '2020-05-15T00:00:00.000Z',
    'SYSTEM_DAT': '2020-05-15T00:00:00.000Z',
    'OBJECTID':

In [16]:
recent_df.to_csv("../recent_phu.csv")

In [29]:
with open("../Ministry_of_Health_Public_Health_Unit_Boundary Simplified.json") as f:
    phu_data = json.load(f)

fig = px.choropleth_mapbox(recent_df, geojson=phu_data, featureidkey='properties.PHU_ID', locations='PHU_NUM', color='ACTIVE_CASES',
                           color_continuous_scale="Viridis",
                           range_color=(0, 4000),
                           mapbox_style="carto-positron",
                           zoom=4, center = {"lat": 48.31, "lon": -84.73},
                           opacity=0.5,
                           labels={'PHU_NUM': 'PHU Number'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [29]:
#df.groupby('kind').agg(min_height=('height', 'min'), max_weight=('weight', 'max'))

In [9]:
aggs = {
    'ACTIVE_CASES': 'sum',
    'RESOLVED_CASES': 'sum',
    'DEATHS': ['sum', 'count']
}
census_phu = df.groupby(['PHU_NUM']).agg(aggs)
census_phu

ACTIVE_CASES RESOLVED_CASES  DEATHS      
                 sum            sum     sum count
PHU_NUM                                          
2226            1009           9368       0   290
2227           10194          75632    1525   290
2230           68316         651640   49413   290
2233            4244          44280       2   290
2234            9819         114881    9868   290
2235            4505          62049    5816   290
2236           62781         422933   10962   290
2237           64554         416479   16671   290
2238            2612          20411    1331   290
2240            7171          83690     596   290
2241            4820          43422       4   290
2242            9006          87844    6681   290
2243            6643          95339   14075   290
2244           43346         247016   15987   290
2246           44486         321014   20294   290
2247            1099          12073     246   290
2249            1580          16295       0   290
2251          109847        1072158   76529   290
2253          442555        3004587   85563   290
2255            3614          35392     726   290
2256            1355          20051    2186   290
2257            2014          18370     289   290
2258           18173          99866    5048   290
2260           37402         284839   10868   290
2261            3101          30895     546   290
2262            5717          40947     838   290
2263             770           5676       2   290
2265           51883         512628   31447   290
2266           28089         202773    9805   290
2268           90913         647100   21419   290
2270          171027        1393859   69251   290
3895          637180        5245088  322611   290
4913           14604          79737    1777   290
5183            6108          43201    2285   290

In [10]:
census_phu.columns = ["_".join(x) for x in census_phu.columns]

In [11]:
census_phu = census_phu.join(hm)
census_phu

,ACTIVE_CASES_sum,RESOLVED_CASES_sum,DEATHS_sum,DEATHS_count,PHU_NAME,0
PHU_NUM,,,,,,
2226,1009,9368,0,290,ALGOMA DISTRICT,290
2227,10194,75632,1525,290,BRANT COUNTY,290
2230,68316,651640,49413,290,DURHAM REGION,290
2233,4244,44280,2,290,GREY BRUCE,290
2234,9819,114881,9868,290,HALDIMAND-NORFOLK,290
2235,4505,62049,5816,290,"HALIBURTON, KAWARTHA, PINE RIDGE",290
2236,62781,422933,10962,290,HALTON REGION,290
2237,64554,416479,16671,290,CITY OF HAMILTON,290
2238,2612,20411,1331,290,HASTINGS & PRINCE EDWARD COUNTIES,290


In [15]:
census_phu['ACTIVE_CASES_sum'].sum() + census_phu['RESOLVED_CASES_sum'].sum() + census_phu['RESOLVED_CASES_sum'].sum()

32893603